# CSE3009 - Internet Of Things Project

## RNN model for Humidity level prediction

Team members-
1. Hartej Singh (19BCT0168)
2. Aniruddha Sharma (19BCT0166)
3. Saurabh Dahale (19BCT0179)
4. Parth Maitrey (19BCT0188)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv(r"D:\VIT\Semester-5\B1 CSE3009 (IoT)\Project\archive\iot_telemetry_data.csv")
x = 10000
df = []
for i in range(len(dataset["device"])):
    if dataset.device[i] == "1c:bf:ce:15:ec:4d":
        df.append(dataset.humidity[i])
train_set = df[:x+1]
train_set = np.array(train_set).reshape(-1,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
scaled_training = sc.fit_transform(train_set)
scaled_training.size

In [ ]:
x_train = []
y_train = []
days = 30
size = scaled_training.size
 
for i in range(days, size):
    x_train.append(scaled_training[i-days:i, 0])
    y_train.append(scaled_training[i,0])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.ndim, y_train.ndim)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

model = Sequential()
model.add(LSTM(units=days, return_sequences=True, input_shape=(days,1))) 
model.add(Dropout(0.2))
model.add(LSTM(units=days, return_sequences=True, input_shape=(days,1))) 
model.add(Dropout(0.2))
model.add(LSTM(units=days, return_sequences=True, input_shape=(days,1))) 
model.add(Dropout(0.2))
model.add(LSTM(units=days)) 
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae", "mse"])
model.fit(x_train, y_train, epochs=10)

In [ ]:
# dataset_test = pd.read_csv(r"D:\VIT\Semester-5\B1 CSE3009 (IoT)\Project\archive\iot_telemetry_data.csv")
# y_test = df[x+1:x+1001]
y_test = df[x+1:x+1001]
y_test = np.array(y_test).reshape(-1,1)
print(y_test.shape)
y_test[29]

In [ ]:
# inputs = dataset_test.iloc[:, 3:4].values
# inputs = df[x+1:]
inputs = df[x+1:]
inputs = np.array(inputs)
inputs = inputs.reshape(-1,1)
inputs = sc.fit_transform(inputs)
inputs.shape

In [ ]:
x_test = []
y_test2 = []
for i in range(days, days+1000):
    x_test.append((inputs[i-days:i , 0]))
    y_test2.append((inputs[i,0]))
x_test = np.array(x_test)
print(x_test.shape)
x_test = np.reshape(x_test, (1000,days,1))
x_test[29]

In [ ]:
y_pred = model.predict(x_test)
y_pred = sc.inverse_transform((y_pred))
y_pred.shape

In [ ]:
from sklearn.metrics import r2_score
acc = r2_score(y_pred, y_test)
print(acc)

In [ ]:
plt.rcParams["figure.figsize"]=10,10
plt.plot(y_test, color = 'red', label="actual")
plt.plot(y_pred, color = 'blue', label="predicted")
plt.legend()
plt.savefig(r"D:\VIT\Semester-5\B1 CSE3009 (IoT)\Project\R2\AI_model_output", dpi=100, bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
import random
temp = []
for i in range(0,30):
    temp.append(random.randint(50, 85))
print(temp)

In [ ]:
temp = np.array(temp)
temp = temp.reshape(-1,1)
temp = sc.fit_transform(temp)
temp = np.reshape(temp, (1,30,1))
print(temp.shape)

In [ ]:
y_temp = model.predict([temp])
y_temp

In [ ]:
y_temp = sc.inverse_transform(y_temp)
y_temp

In [ ]:
model.save("RNNmodel-Humidity.h5")

In [ ]:
import joblib
joblib.dump(sc, "scaler")